In [1]:
from bs4 import BeautifulSoup
import pandas as pd
import requests
import numpy as np
from time import sleep

In [2]:
## Pass in a URL and it returns the data for that professor
def get_data(url):
    req = requests.get(url).content
    bs = BeautifulSoup(req)
    name = list(bs.find("div", class_="NameTitle__Name-dowf0z-0 cfjPUG").descendants)
    name = f"{name[1]} {name[4]}"
    department = bs.find("a", class_="TeacherDepartment__StyledDepartmentLink-fl79e8-0 fuypDB").b.contents[0]
    school = bs.find("a", href="/school?sid=1074").contents[0]
    tags = np.unique(bs.find_all("span", class_="Tag-bs9vf4-0 hHOVKF"), return_counts=True)
    tags = {i:j for i,j in zip(tags[0], tags[1])}
    rating = bs.find("div", class_="RatingValue__Numerator-qw8sqy-2 liyUjw").contents[0]
    feedback = bs.find_all("div", "FeedbackItem__FeedbackNumber-uof32n-1 kkESWs")
    take_again = feedback[0].contents[0][:-1]
    difficulty = feedback[1].contents[0]
    return [name, department, school, tags, rating, take_again, difficulty]


In [3]:
## Example dataframe -> appends data to csv file
pd.DataFrame(data=get_data("https://www.ratemyprofessors.com/professor?tid=2466852"), index=["name", "department", "school", "tags", "rating", "take_again", "difficulty"]).transpose().to_csv(f"professors.csv", mode='a', header=False, index=False)

In [ ]:
## While Loop that checks random pages for professors -> appends data to csv file

count = 0
page = 2466852
while count < 10:
    try:
        pd.DataFrame(data=get_data(f"https://www.ratemyprofessors.com/professor?tid={page}"), index=["name", "department", "school", "tags", "rating", "take_again", "difficulty"]).transpose().to_csv(f"professors.csv", mode='a', header=False, index=False)
        count += 1
        print('x')
    except Exception as e:
        print(e)
    finally:
        page += 12